1\. Write a function that converts number representation (bin<->dec<->hex)

In [11]:
def f(n):
    print()
    print('Decimal representation is:', n)
    print('Binary representation of:', bin(n))
    print('Hexadecimal representation :', hex(n))    
f(100)
f(0b1100101) #0b indicates binary
f(0x66)      #0x indicates hexadecimal
#the function take a number in a representetion and print its conversion


Decimal representation is: 100
Binary representation of: 0b1100100
Hexadecimal representation : 0x64

Decimal representation is: 101
Binary representation of: 0b1100101
Hexadecimal representation : 0x65

Decimal representation is: 102
Binary representation of: 0b1100110
Hexadecimal representation : 0x66


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [2]:
def converter(n):
    segno=int(n[0])
    esponente=int("0b"+n[1:9],2) -127
    mant="1"+n[9:32]
    t=0
    mantissa=0
    for i in mant:
        mantissa+=int(i)* 2**(-t)
        t=t+1
    if segno==1 :   
        print(-mantissa**(esponente))
    elif segno==0:
        print(mantissa*2**(esponente))
    else: 
        print("your string is wrong")
#please do not insert special number    
converter("01000001110010000000000000000000") 
converter("11000001111110000010000000001111") 

25.0
-14.120296633298372


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [3]:
under=1.
over=1.

while over*2!= float('inf') :
    over=over*2
print("overflow occurs after",over)    

while  under/2!= float(0) :
    under=under/2
print("underflow occurs after",under)  



    

overflow occurs after 8.98846567431158e+307
underflow occurs after 5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [4]:
epsilon=1.
epslast=0
while 1 != 1 + epsilon:
    epslast=epsilon
    epsilon=epsilon/2
print("the machine precision is: ",epslast)

the machine precision is:  2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [3]:
import math
def quadratic1(a,b,c):
    if(a == 0): # bx+c=0
        if(b != 0):
            x=-c/b
            print("La soluzione è x =", x)
        else:
            if(c == 0):
                print("Indeterminata")
            else:
                print("Impossibile")
    else:
        delta=b*b-4*a*c
        if(delta > 0):
#case a) 
#            x1=(-b-math.sqrt(delta))/(2*a)
#            x2=(-b+math.sqrt(delta))/(2*a)
#x1 = -999999.999999
#x2 = -9.999894245993346e-07
          
#case b)    
            x2=2*c / (-b-math.sqrt(delta))
            x1=2*c / (-b+math.sqrt(delta))
#

#x1 = -1000010.5755125057
#x2 = -1.000000000001e-06

#There is a slightly difference between the two solution due to the choise of the parameters
#with this choice we have problems due to the fact that computers store numbers in binary format
#for the same reason if we print(0.01/0.1) I get 0.09999999999999999 instead of 0.1

            print("x1 =", x1)
            print("x2 =", x2)
        elif(delta == 0):
            x=-b/a
            print("x =", x)
        else:
            xr=-b/(2*a)
            xi=math.sqrt(-delta)/(2*a)
            x1=complex(xr,xi)
            x2=complex(xr,-xi)
            print("x1 =", x1)
            print("x2 =", x2)
quadratic1(0.001,1000,0.001)

x1 = -1000010.5755125057
x2 = -1.000000000001e-06


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:
import math
def der(x,delta):
#    meno preciso    
#    derivata = ((x+delta)*((x+delta) - 1) - x*(x-1))**2 / ((delta)**2)
    derivata2 = ( (x+delta)**2- (x+delta) - x*(x-1))    /(delta)
    print("per delta = ",delta, "la derivata :", derivata2)

der(1, 10**(-2))
der(1, 10**(-4))
der(1, 10**(-6))
der(1, 10**(-8))
der(1, 10**(-10))
der(1, 10**(-12))
der(1, 10**(-14))
#the analytical solution is 1, we can see that:
#as we increase delta
#the approximation error become smaller, instead  
#the roundoff error increase

per delta =  0.01 la derivata : 1.0099999999999998
per delta =  0.0001 la derivata : 1.0000999999992821
per delta =  1e-06 la derivata : 1.000001000006634
per delta =  1e-08 la derivata : 0.999999993922529
per delta =  1e-10 la derivata : 1.000000082740371
per delta =  1e-12 la derivata : 1.000088900582341
per delta =  1e-14 la derivata : 0.9992007221626409


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [6]:
import math as m
import time 
#(a)
def integ(n):
    h=2/n
    s=0
    i=-1
    while(i<=1):
         s+= h*m.sqrt(1-i**2)
         i=i+h
    return s
integral = integ(100)
print("The integral for N = ", 100, "is:", integral)
print("The difference with the true value is:",abs(m.pi/2 - integral))

#(b)
#1s

#I give a starter number of cycle
n=1.9e+06
tempo=0
nfinal=0
while(tempo <1.0):
    nfinal=n
    start= time.process_time()
    integ(n)
    tempo=time.process_time() - start
    n=n+10000
    
print("In 1s the max N is around:", nfinal)   


The integral for N =  100 is: 1.5691342555492493
The difference with the true value is: 0.001662071245647212
In 1s the max N is around: 3170000.0


In [7]:
#last output on my pc is : In 1s the max N is around: 3170000.0

#1min
nfinal=3170000*60# assuming it is linear  
start= time.process_time()
inte=integ(nfinal)
tempo=time.process_time() - start
print("In", tempo," the max N is around:", nfinal, " the difference with the true value is",abs(m.pi/2 - inte) )   

#my pc's output is: 
#In 58.92644248500005s the max N is around: 190200000  the difference with the true value is 1.1079313022577253e-09
#we gain 6 order of precision


#while(tempo<60): #tooks ages
#    nfinal=n
#    start= time.process_time()
#    integ(n)
#    tempo=time.process_time() - start
#    n+=n
#print("In 1m the max N is around:", nfinal)   

58.92644248500005
In 60s the max N is around: 190200000  the difference with the true value is 1.1079313022577253e-09
